In [ ]:
!pip install -q fitparse


In [ ]:
# @title Import libraries
# @markdown This cell loads libraries that we will use througout the notebook.
import io
import os
import codecs

import altair as alt
import numpy as np
import pandas as pd

from timesketch_api_client import config
from timesketch_import_client import helper
from timesketch_import_client import importer

import fitparse


In [ ]:
# Load the FIT file
file_name = '2019-02-17-062644-ELEMNT-297E-195-0.fit'
path = './scripts/garmin/python-fitparse/tests/files/'

file = path+file_name
fitfile = fitparse.FitFile(file) # for testing purposes we will loop the full folder later

In [ ]:
pd.set_option('max_colwidth', 500)

In [ ]:
ts_client = config.get_client()
[(x.id, x.name) for x in ts_client.list_sketches()]

In [ ]:
# @markdown This needs to be changed to reflect the correct sketch.

SKETCH_ID = 2 # @param {type: "integer"}

In [ ]:
def process_file(fitfile,filename):
  # Iterate over all messages of type "record"
  # (other types include "device_info", "file_creator", "event", etc)
  all_entries = []
  headers = []
  device_sting = ""
  for device in fitfile.get_messages("device_info"):
    #print(device.get_values())
    #device_sting = " ".join((device_sting, str(device.get_values())))
    all_entries.append(device.get_values())

  for record in fitfile.get_messages("record"):
      recordt_row = []
      entry = record.get_values()
      #if 'device_info' not in entry.keys():
      #  entry['device_info'] = device_sting
      recordt_row.append(record.get_values())
      #entry.append("device_info")
      #print(entry)
      # Records can contain multiple pieces of data (ex: timestamp, latitude, longitude, etc)
      all_entries.append(entry)

  df = pd.DataFrame(all_entries)
  if 'timestamp' in df:
    df['datetime'] = pd.to_datetime(df['timestamp'])
  else:
    return
  return df

  

In [ ]:
def ts_upload_file(df,filename):
  sketch = ts_client.get_sketch(SKETCH_ID)
  import_helper = helper.ImportHelper() 

  with importer.ImportStreamer() as streamer:
    streamer.set_sketch(sketch)
    streamer.set_config_helper(import_helper) 
    streamer.set_timeline_name(filename)
    streamer.add_data_frame(df)

In [ ]:
import os
import time

for filename in os.listdir(path):
    if filename.endswith(".fit"):
        file = path+filename

        fitfile = fitparse.FitFile(file) 
        print(file)
        return_df = process_file(fitfile,filename)
        print("done, now uploading")
        ts_upload_file(return_df,filename)
        print("donw uploading")
        time.sleep(1)
        continue
    else:
      continue

# Analysis

In [ ]:
sketch = ts_client.get_sketch(2)
cur_df = sketch.explore(
    'speed:*',
    as_pandas=True,
    return_fields='datetime,speed'
)

In [ ]:
cur_df.head(100)

In [ ]:
df.shape

In [ ]:
!pip install matplotlib
!pip install seaborn

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import seaborn as sns

sns.set(rc={'figure.figsize':(11, 4)})


In [ ]:
cur_df['speed'].plot(linewidth=0.5);


In [ ]:
sketch = ts_client.get_sketch(2)
cur_df = sketch.explore(
    'position_lat:*',
    as_pandas=True,
    return_fields='position_lat,position_long'
)

In [ ]:
cur_df

In [ ]:
!pip install geos
!pip install -q geopy
!pip install -q folium


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:,.20f}'.format



In [ ]:
cur_df['lat'] = cur_df['position_lat'] /100000000
cur_df['lon'] = cur_df['position_long']/100000000
cur_df.info()
cur_df

In [ ]:
df_new = cur_df.head(100)
df_new2 = cur_df.drop_duplicates(subset=['lon'])

In [ ]:
!pip install -q  folium
import pandas as pd
import folium

In [ ]:
# demo that it would work
df_new = pd.DataFrame(columns=["lat", "lon"], data=[[56.37772540,102.740],[58.37772540,103.740]]),
df_new = pd.DataFrame({
    'lat': [56.37772540, 58.37772540,-52.850,68.462,49.6633487,56.37],
    'lon': [102.740, 103.740,68.462,-5.2850,8.2508508,10.2740]
})


new = df_new


In [ ]:
df_new.head(19)

In [ ]:

map1 = folium.Map(location=(49.850,	8.2462), zoom_start=3)
for index,row in df_new.iterrows(): 
  # Add the geocoded locations to the map
  folium.Marker(location=(row['lat'],row['lon']),popup='bla').add_to(map1)

display(map1)